# Ejemplo Web scrapping 2: Noticias deportivas

En este notebook vamos a ver otro ejemplo de web scrapping, este con noticias deportivas:

## Importando librerías

Al igual que en el notebook anterior, importamos lo necesario:

In [52]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


from urllib.request import urlopen

Leemos la web:

In [56]:
# Cabeceras:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
# url web a scrapear:
marca = "https://www.marca.com/"

response = get(marca, headers = headers)

Hacemos el get:

In [57]:
print(response) # un 200 es una buena señal :D

<Response [200]>


Todo bien, pasamos a sacar cosas:

In [60]:
html_soup = BeautifulSoup(response.text, 'html.parser')

Tras enalizar la web, nos hemos fijado que todas las noticias tienen etiquetas ``h2`` y clase ``mod-title``:

In [64]:
noticias = html_soup.find_all('h2', class_='mod-title')
noticias[0]

<h2 class="mod-title" itemprop="headline"><a href="https://www.marca.com/futbol/real-madrid/2021/01/21/600997a0e2704e87388b4585.html" itemprop="url" title="La charla de Zidane tras el bochorno: los jugadores esperaban otras palabras"> La charla de Zidane tras el bochorno: los jugadores esperaban otras palabras
</a></h2>

Dentro de cada una de ellas, tenemos que extraer, a su vez, el ``title`` de las ``a``:

In [73]:
noticia_ejemplo = noticias[0].find_all("a")[0]
# Con el strip eliminamos todas aquellas cosas raras al comienzo y final del string:
noticia_ejemplo.text.strip()

'La charla de Zidane tras el bochorno: los jugadores esperaban otras palabras'

También podemos sacar el enlace como:

In [76]:
noticia_ejemplo.get_attribute_list('href')[0]

'https://www.marca.com/futbol/real-madrid/2021/01/21/600997a0e2704e87388b4585.html'

Ahora podemos acceder a ese link y extraer la información de la noticia en sí:

In [80]:
url_2 = noticia_ejemplo.get_attribute_list('href')[0]

response_2 = get(url_2, headers = headers)
html_soup_2 = BeautifulSoup(response_2.text, 'html.parser')


# for texto in html_soup_2.find_all("p"):
#     print(texto.text)

# El texto que queremos está en los p y en los h3, pero también se nos está colando cosas que no nos resultan neceasrias, ¿cómo podemos proceder?
# Pues leyendo todo junto (que estará en orden según la página) y quedándonos con el texto hasta que detectemos algo que nos haga parar.
# En este caso, ese algo será cuando lleguemos a un texto 'h3' que diga "Temas relaciondos". A partir de ese momento, ya no nos interesan 
#ni los 'p' ni los 'h3'
texto_noticia = html_soup_2.find_all(["p", "h3"])
texto_noticia

noticia_completa = []
cont = 0
for line in texto_noticia:
    cont += 1
    if line.text == 'Temas relacionados':
        break
    # Nos saltamos siempre las 2 primeras líneas:
    if cont > 2:
        noticia_completa.append(line.text.strip())

# noticia_completa
# Y lo juntamos todo en un solo texto:
''.join(noticia_completa)

'Zidane sigue en el cargoReal Madrid en crisis, noticias de última horaNo llegó a dos minutos, sin ninguna palabra más alta que la otra, en la misma línea de su discurso ante la prensa tras el K.O. contra el Alcoyano. "Esto no es una vergüenza", señaló Zidane. Y apenas doce horas después, sin tiempo para dormir y descansar como se debe, continuó el mismo relato ante un vestuario sorprendido, donde había quienes esperaban, y deseaban, palabras más duras, una bronca para espabilar, reproches lógicos después de una eliminación ante un club cuyo presupuesto es menor que cualquier ficha del primer equipo.Zinedine apeló a la unidad, animó a los jugadores a salir de esta, después de dos malos partidos (un adjetivo generoso) y recalcó: "Todos podemos dar un poco más". Si bien la plantilla ya conoce a Zidane y saben que no es un entrenador de mano dura, sí se entendía en parte del grupo que era día para elevar el tono, huir del gris y no ahorrar calificativos. Agitar a un equipo que de los últi

In [81]:
texto_noticia

[<p class="edition-selector__menu-title">Ediciones:</p>,
 <p class="tab-news-title js-newsTitle">Futbol/real-madrid</p>,
 <p class="summary-lead second izquierda"><i class="item-sumary-lead"></i><a href="https://www.marca.com/futbol/real-madrid/2021/01/21/6009747be2704e682d8b45b0.html">Zidane sigue en el cargo</a></p>,
 <p class="summary-lead second izquierda"><i class="item-sumary-lead"></i><a href="https://www.marca.com/futbol/real-madrid/2021/01/21/600984e2268e3e3f218b4580.html">Real Madrid en crisis, noticias de última hora</a></p>,
 <p><span class="capital-letter">N</span>o llegó a <strong>dos minutos</strong>, sin ninguna palabra más alta que la otra, en la misma línea de su discurso ante la prensa tras el K.O. contra el Alcoyano. <strong><a href="https://www.marca.com/futbol/copa-rey/2021/01/20/6008a46722601d163f8b45bd.html" target="_blank">"Esto no es una vergüenza", señaló Zidane.</a></strong> Y apenas doce horas después, sin tiempo para dormir y descansar como se debe, contin

Iteramos con esta condición. Y para combinarlo con lo otro, podríamos extraer todos los links a noticias y, posteriormente, extraer el texto de cada una de ellas:

In [82]:
noticias[0].find_all("a")[0].text.strip()

'La charla de Zidane tras el bochorno: los jugadores esperaban otras palabras'

In [83]:
# Leemos títulos de artículos y sus links y los guardamos:
marca = "https://www.marca.com/"

response = get(marca, headers = headers)

html_soup = BeautifulSoup(response.text, 'html.parser')

noticias = html_soup.find_all('h2', class_='mod-title')

# Noticia a noticia para sacar links:
textos = []
links = []
for noticia in noticias:
    try:
        text_noticia = noticia.find_all("a")[0].text.strip()
        link = noticia.find_all("a")[0].get_attribute_list('href')[0]
        
        textos.append(text_noticia)
        links.append(link)
    except:
        pass
    
# Una vez tenemos las noticias, iteramos por cada uno de sus enlaes:
textos_noticias_completos = []
for link in links:
    response_2 = get(link, headers = headers)
    html_soup_2 = BeautifulSoup(response_2.text, 'html.parser')
    
    texto_noticia = html_soup_2.find_all(["p", "h3"])
    texto_noticia
    noticia_completa = []
    cont = 0
    for line in texto_noticia:
        cont += 1
        if line.text == 'Temas relacionados':
            break
        # Nos saltamos siempre las 2 primeras líneas:
        if cont > 2:
            noticia_completa.append(line.text.strip())
    
    textos_noticias_completos.append(''.join(noticia_completa))

Finalmente, nos creamos el dataframe con todo lo que tenemos.

Importante, si vamos a manejar muchos datos (pero muchos muchos), será mejor ir creando el dataframe poco a poco (cada x llamadas) para no almacenar demasiada información duplicada al mismo tiempo.

In [84]:
cols = ['Título', 'Descripción', 'Enlace']

noticias_deportivas = pd.DataFrame({'Título': textos,
                                    'Descripción': textos_noticias_completos,
                                    'Enlace': links})[cols]

noticias_deportivas.to_excel('marca_noticias.xls', index=False)
noticias_deportivas_df = pd.read_excel('marca_noticias.xls')

In [85]:
noticias_deportivas_df

,Título,Descripción,Enlace
0,La charla de Zidane tras el bochorno: los juga...,"Zidane sigue en el cargoReal Madrid en crisis,...",https://www.marca.com/futbol/real-madrid/2021/...
1,Ya rueda el balón en Mestalla,Narrador:El Valencia carga el ataque por la ba...,https://www.marca.com/futbol/laliga-santander/...
2,Arranca el partido en Ibiza,"Narrador:Buena acción individual de Lekue, lle...",https://www.marca.com/futbol/copa-del-rey/ibiz...
3,The Times desvela el borrador de la Superliga:...,Estos son los motivos por los que la FIFA ha e...,https://www.marca.com/futbol/champions-league/...
4,"Juanfran: ""Que hablen mierda de mí, que se ría...",El Inter 'humilla' al Sao Paulo de Juanfran y ...,https://www.marca.com/futbol/america/2021/01/2...
5,Durísimo palo de FIFA a la Superliga para acab...,Reunión de Florentino con Agnelli en TurínPreg...,https://www.marca.com/futbol/futbol-internacio...
6,"""Fernando y Michael me metieron el amor por el...","Esteban Ocón, compañero de Fernando Alonso est...",https://www.marca.com/motor/rallies/2021/01/21...
7,"""Si se escapa un alarido es que marca el Alcoy...",Parodia playera del mítico calentamiento de Ma...,https://videos.marca.com/v/0_fd5229mr-premonic...
8,Así son las dos personas más guapas del mundo,NaN,https://www.marca.com/tiramillas/actualidad/al...
9,Se avecina revolución blanca: la estabilidad d...,Heurtel se libera del Barça... y apunta al Mad...,https://www.marca.com/baloncesto/acb/2021/01/2...
